In [7]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

In [8]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
filename='./configs/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco_balanced.py'
cfg = Config.fromfile(filename)

root='../../dataset/'

epoch = 'latest'

# dataset config 수정
cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 4

cfg.seed=2021
cfg.gpu_ids = [1]
cfg.work_dir = './work_dirs/cascade_rcnn_r50_fpn_1x_coco_balanced'

# change number of classes
if(filename in ['./configs/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco_balanced.py',
                './configs/convnext/cascade_mask_rcnn_convnext-t_p4_w7_fpn_giou_4conv1f_fp16_ms-crop_3x_coco.py']):
    cfg.model.roi_head.bbox_head[0].num_classes=10
    cfg.model.roi_head.bbox_head[1].num_classes=10
    cfg.model.roi_head.bbox_head[2].num_classes=10
elif(filename in ['./configs/swin/retinanet_swin-t-p4-w7_fpn_1x_coco.py',
                     './configs/detr/detr_r50_8x2_150e_coco.py']):
    cfg.model.bbox_head.num_classes = 10
elif(filename in ['./configs/yolox/yolox_tiny_8x8_300e_coco.py']):
    cfg.model.bbox_head.feat_channels=10
elif(filename in ['./configs/yolo/yolov3_d53_320_273e_coco.py']):
    pass
else:
    cfg.model.roi_head.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

In [9]:
# build dataset & dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [4]:
# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'latest.pth')

model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

load checkpoint from local path: ./work_dirs/cascade_rcnn_r50_fpn_1x_coco_balanced/latest.pth


In [5]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05) # output 계산

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4871/4871, 14.7 task/s, elapsed: 331s, ETA:     0s

In [12]:
# submission 양식에 맞게 output 후처리
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
img_ids = coco.getImgIds()

class_num = 10
for i, out in enumerate(output):
    # 2d-array 형식, bbox 좌표
    # output[[plastic-arr], [paper-arr], ...]
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
[5, 56, 0, 0, 0, 1, 2, 36, 0, 0]


RuntimeError: No active exception to reraise

'1.13.1+cu117'

True
